# **Xây dựng RAG với LangChain**

In [ ]:
# !pip install \
#   langchain-chroma==0.1.4 \
#   langchain_community \
#   langchain-core==0.3.15 \
#   langchain-openai==0.2.6 \
#   langchain-text-splitters==0.3.2 \
#   python-dotenv==1.0.1 \
#   pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.

In [ ]:
# !pip install -qU \
#   langchain_huggingface \
#   transformers \
#   sentence-transformers

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Load file**

In [ ]:
# Read in State of the Union Address File
# file_path = "/content/drive/MyDrive/2025B/Tiểu luận Sinh Promela/Data/spinPrimer.pdf"
file_path = "/content/spinPrimer.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()
print(docs[0].page_content[:930])


[ Team LiB ]
• Table of Contents
Spin Model Checker, The: Primer and Reference Manual
By Gerard J. Holzmann
Publisher: Addison Wesley
Pub Date: September 04, 2003
ISBN: 0-321-22862-6
Pages: 608
SPIN is the world's most popular, and arguably one of the world's most powerful, tools for detecting software defects
in concurrent system designs. Literally thousands of people have used SPIN since it was first introduced almost fifteen
years ago. The tool has been applied to everything from the verification of complex call processing software that is
used in telephone exchanges, to the validation of intricate control software for interplanetary spacecraft. 
This is the most comprehensive reference guide to SPIN, written by the principal designer of the tool. It covers the
tool's specification language and theoretical foundation, and gives detailed advice on methods for tackling the most
complex software verification problems


**Initialize Embeddings Model and Vector Store**

In [ ]:
# Get Embeddings Model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Initialize ChromaDB as Vector Store
vector_store = Chroma(
    collection_name="spin_primer",
    embedding_function=embeddings
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


**Split the File into LangChain Documents & Save to Vector Store**

In [ ]:
# Initialize Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# Create Documents (Chunks) From File
splits = text_splitter.split_documents(docs)

# Save Document Chunks to Vector Store
ids = vector_store.add_documents(splits)

# Query the Vector Store
results = vector_store.similarity_search(
    'Promela',
    k=2
)

# Print Resulting Chunks
for res in results:
    print(f"* {res.page_content} [{res.metadata}]\n\n")


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


* [ Team LiB ]
Finding Out More
 This concludes our overview of the main features of the PROMELA specification language. A few more seldomly
used constructs were only mentioned in passing here, but are discussed in greater detail in the manual pages that are
included in Chapters 16 and 17. More examples of PROMELA models are included in Chapters 14 and 15. A
definition of the operational semantics for PROMELA can be found in Chapter 7.
 Alternate introductions to the language can be found in, for instance, Ruys [2001] and Holzmann [1991]. Several
other tutorial-style introductions to the language can also be found on the SPIN Web site (see Appendix D).
[ Team LiB ] [{'page': 93, 'source': '/content/spinPrimer.pdf'}]


* [ Team LiB ]
Examples
 To get started, we discuss a few small examples of PROMELA specifications. We will prompt you for the things that
are worth observing in these models, and for some experiments you can do to explore them further. We do not intend
to define the lang

# **RAG Pipeline**

In [ ]:
# Create Document Parsing Function to String
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Set Chroma as the Retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 4})


In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline

pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    #model="google/flan-t5-large",
    max_new_tokens=512
)

# Initialize the LLM instance
llm = HuggingFacePipeline(pipeline=pipe)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
from langchain_core.prompts import PromptTemplate
# Create the Prompt Template
prompt_template = """
Use the context provided to answer the question.
If the answer is not contained in the context, say you do not know.

Context:
{context}

Question:
{query}

Answer:
"""

custom_rag_prompt = PromptTemplate.from_template(prompt_template)


In [ ]:
# Create the RAG Chain
rag_chain = (
    {
        "context": retriever | format_docs,
        "query": RunnablePassthrough()
    }
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)


In [ ]:
prompt = """
### SYSTEM ROLE
You are an expert Formal Methods Engineer specializing in the SPIN Model Checker and Promela. Your goal is to translate user requirements into syntactically correct, optimized, and verifiable Promela models.

### CONTEXT & KNOWLEDGE (RETRIEVED DATA)
Use the following retrieved context (documentation, examples, or similar patterns) to answer the user request.
---
{{RETRIEVED_CHUNKS}}
---

### INSTRUCTIONS FOR CODE GENERATION
1. **Strict Syntax Compliance:**
   - Ensure all guarded commands (`if`, `do`) utilize `::` correctly.
   - Distinctly differentiate between `mtype` (message types) and variables.
   - Ensure all channels (`chan`) are correctly typed and instantiated.

2. **State Space Optimization (CRITICAL):**
   - Promela is for verification, not just execution. You MUST prioritize state-space reduction.
   - Use `atomic { ... }` blocks for sequences of statements that do not block, to minimize interleavings.
   - Use `d_step { ... }` only for deterministic, non-blocking sequences that modify local variables.
   - Prefer `byte` over `int` or `short` where possible to save memory.

3. **Concurrency & Correctness:**
   - Explicitly handle non-determinism. If multiple guards are true, explain which path SPIN might take.
   - Prevent Deadlocks: Ensure loops have an `else` or explicit exit condition if required.
   - Prevent Livelocks: Ensure progress states are reachable.

4. **Verification Elements:**
   - Include `assert(...)` statements to check safety properties (invariants).
   - If requested, generate `ltl` formulas for temporal properties (Liveness/Safety).
   - Use `active proctype` for processes that should start immediately.

### OUTPUT FORMAT
- Provide the Promela code in a distinct code block.
- Follow the code with a brief "Verification Strategy" explaining:
  - Why you chose specific atomic/d_step blocks.
  - Potential race conditions you mitigated.
  - The expected behavior of the LTL formula (if applicable).

PROMELA code:

active proctype P() {
    int x = 0;
    x++;
}

"""

In [ ]:
# prompt = """
# what is proctype

# """
respone = rag_chain.invoke(prompt)


In [ ]:
respone

'for declaring new process behavior. Syntax proctype name ( [ decl_lst] )  sequence  D_proctype name ( [ decl_lst ] )  sequence  Description All process behavior must be declared before it can be instantiated. The proctype construct is used for the declaration. Instantiation can be done either with the run operator, or with the prefix active that can be used at the time of declaration. Declarations for local variables and message channels may be placed anywhere inside the proctype body. In all cases, though, these declarations are treated as if they were all placed at the start of the proctype declaration. The scope of local variables cannot be restricted to only part of the proctype body. The keyword D_proctype can be used to declare process behavior that is to be executed completely deterministically. If non-determinism is nonetheless present in this type of process definition, it is resolved in simulations in a deterministic, though otherwise undefined, manner. During verifications 

In [ ]:
# https://spinroot.com/spin/Man/Manual.html
# https://spinroot.com/spin/Man/Quick.html

- Promela chuẩn systex

- Nhắc rule SPIN cụ thể

- Đưa ví dụ đã chạy được vào prompt

=> Model ít hallucinate, sửa lỗi nhanh hơn, ít vòng reflection hơn.